In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

load data set

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='A:/chchdata/data/', train=True, download=True, transform = transform)
trainloader = DataLoader(trainset, batch_size=8, shuffle = True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='A:/chchdata/data/', train = False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=8, shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat','deer','dog','frog','horse','ship','truck')

100.0%


Extracting A:/chchdata/data/cifar-10-python.tar.gz to A:/chchdata/data/
Files already downloaded and verified


In [6]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):  # nn.modules를 상속 받는다
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # 클래스 10개
    
    def forward(self, x): # x가 input
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5) # 일자로 피는 작업
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) 
for epoch in range(1) :
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0): 
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimizer
        outputs = net(inputs)
        loss = criterion(outputs, labels) # crossentropyloss
        loss.backward()
        optimizer.step() # 옵티마이저 계산

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999 :
            print('[%d, %5d] loss: %.3f' % 
            (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
print('Finished Training')

C:\Users\sswwd\anaconda3\envs\chch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,  2000] loss: 2.150
[1,  4000] loss: 1.777
[1,  6000] loss: 1.596
Finished Training


모델 저장

In [8]:
PATH = 'A:\chchdata\h5\cifar_net.pth'  # 모델에서 나온 가중치를 저장할 경로 지정
torch.save(net.state_dict(), PATH) 

load model

In [9]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [10]:
correct = 0
total = 0
with torch.no_grad():   # 추적 안함. 업데이트 x
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1) # 10개의 클래스에서 가장 큰 값으로 예측 ==argmax
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images : %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images : 45 %
